# Data formats for a topological analysis of circuit subvolumes

The topological analysis pipeline archives the computed data to an HDf store using a
schema specified in the pipeline config.
Here we discuss the schema, and a utility that the package provides to load node-properties
and connectivity matrices.


Let us begin by loading the required code, and data

## Virtual Environment

A Python virtual environment is needed to run the notebook,
and for writing your own.
This notebook has been run in an environment created with the posix shell command
`source /gpfs/bbp.cscs.ch/project/proj83/home/sood/venv.sh`

In [1]:
from importlib import reload
from pathlib import Path
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sbn

At of 20211112, the pipeline *sub-package* is called `connsense`, and can be
pip installed from the repository (branch `add-subpackages`.

Reading, and writing of topological analysis data uses `connsense.io`.

In [3]:
from connsense import pipeline
from connsense.io import logging, read_config, write_results

LOG = logging.get_logger("TopoAnalysis DataFormats", "INFO")

LOG.info("A tutorial on interacting with topological analysis pipeline data")

 2021-11-12 13:44:56,392: A tutorial on interacting with topological analysis pipeline data


We already have extracted connectivity data that we can use.

In [4]:
proj83 = Path("/gpfs/bbp.cscs.ch/project/proj83")
path_topo_analysis = proj83 / "analyses" / "topological-analysis-subvolumes"
LOG.info("Configuration and data")
for f in path_topo_analysis.glob('*'):
    LOG.info("\t%s", f)
path_config = path_topo_analysis/"config.json"

 2021-11-12 13:45:03,415: Configuration and data
 2021-11-12 13:45:03,419: 	/gpfs/bbp.cscs.ch/project/proj83/analyses/topological-analysis-subvolumes/topological_sampling.h5
 2021-11-12 13:45:03,420: 	/gpfs/bbp.cscs.ch/project/proj83/analyses/topological-analysis-subvolumes/config.json
 2021-11-12 13:45:03,420: 	/gpfs/bbp.cscs.ch/project/proj83/analyses/topological-analysis-subvolumes/.config.json.swp


In [21]:
reload(pipeline)
config = read_config.read(path_topo_analysis/"config.json")
tap = pipeline.TopologicalAnalysis(path_config)
paths = config["paths"]
LOG.info("Pipeline data HDF-store: \n\t%s", tap._data._root)
for i, (step, group) in enumerate(tap._data._groups.items()):
    LOG.info("\t(%s) %s: %s", i, step, group)

 2021-11-12 13:53:25,645: Pipeline data HDF-store: 
	/gpfs/bbp.cscs.ch/project/proj83/analyses/topological-analysis-subvolumes/topological_sampling.h5
 2021-11-12 13:53:25,646: 	(0) define-subtargets: subtargets
 2021-11-12 13:53:25,647: 	(1) extract-neurons: neurons
 2021-11-12 13:53:25,649: 	(2) evaluate-subtargets: subtarget_quality
 2021-11-12 13:53:25,650: 	(3) extract-connectivity: con_mats/original
 2021-11-12 13:53:25,651: 	(4) randomize-connectivity: con_mats/randomized
 2021-11-12 13:53:25,653: 	(5) analyze-connectivity: analysis


The pipeline object we have defined above can be used to inspect all the data
that has already been computed in the pipeline.

In [22]:
LOG.info("Circuits for which data is available: ")

for  i, c in enumerate(tap.data.circuits):
    LOG.info("(%s). %s", i, c)

 2021-11-12 13:54:39,009: Circuits for which data is available: 
 2021-11-12 13:54:40,305: (0). Bio_M


Only one circuit is available, along with a single connectome (local).
The subtargets are named by their column and row in the flatspace:

In [29]:
subtargets = tap.data.get_subtargets("Bio_M")
LOG.info("Available subtargets for circuit Bio_M: %s", len(subtargets ))
LOG.info("\t They are named (row, column)")
LOG.info("\tFor example column defined from the 10th row, and 7 column: R10;C7")

 2021-11-12 13:59:27,764: Available subtargets for circuit Bio_M: 247
 2021-11-12 13:59:27,768: 	 They are named (row, column)
 2021-11-12 13:59:27,768: 	For example column defined from the 10th row, and 7 column: R10;C7


In [31]:
s = "R0;C10"
nodes_r0c10 = tap.data.get_nodes("Bio_M", s)
LOG.info("Number of nodes in subtarget %s: %s: ", s,  len(nodes_r0c10))
nodes_r0c10.head()

/gpfs/bbp.cscs.ch/home/sood/rsynced/work/project/proj83/topological-analysis-of-subvolumes/connsense/pipeline.py:213: FutureWarning: Passing lists as key for xs is deprecated and will be removed in a future version. Pass key as a tuple instead.
  
 2021-11-12 14:01:13,736: Number of nodes in subtarget R0;C10: 6719: 


gid            x            y            z synapse_class  \
circuit subtarget                                                              
Bio_M   R0;C10      455  5349.881407 -2189.938376 -5461.554050           INH   
        R0;C10      805  5435.330946 -2085.202960 -5411.225766           INH   
        R0;C10     3930  5422.509430 -2121.012844 -5511.005647           INH   
        R0;C10     4303  5441.431546 -1996.386178 -5491.911368           INH   
        R0;C10     5384  5499.207748 -2019.563479 -5524.047901           INH   

                   layer   mtype  
circuit subtarget                 
Bio_M   R0;C10         1  L1_DAC  
        R0;C10         1  L1_DAC  
        R0;C10         1  L1_DAC  
        R0;C10         1  L1_DAC  
        R0;C10         1  L1_DAC

Adjacencies in the circuit have been extracted for each circuit, and
can be loaded using:

In [36]:
adj_r0c10 = tap.data.get_adjacency(circuit="Bio_M", subtarget=s,
                                   connectome="local")
LOG.info("Adjacency %s: %s", s, adj_r0c10.shape)

/gpfs/bbp.cscs.ch/home/sood/rsynced/work/project/proj83/topological-analysis-of-subvolumes/connsense/pipeline.py:227: FutureWarning: Passing lists as key for xs is deprecated and will be removed in a future version. Pass key as a tuple instead.
  """..."""
 2021-11-12 14:03:22,525: Adjacency R0;C10: (6719, 6719)


We can also get all the data for a subtarget:

In [44]:
data_r0c10 = tap.data.get_data(circuit="Bio_M", subtarget="R0;C10")

LOG.info("Data availabel for column R0;C10: %s", list(data_r0c10.keys()))

/gpfs/bbp.cscs.ch/home/sood/rsynced/work/project/proj83/topological-analysis-of-subvolumes/connsense/pipeline.py:213: FutureWarning: Passing lists as key for xs is deprecated and will be removed in a future version. Pass key as a tuple instead.
  
/gpfs/bbp.cscs.ch/home/sood/rsynced/work/project/proj83/topological-analysis-of-subvolumes/connsense/pipeline.py:227: FutureWarning: Passing lists as key for xs is deprecated and will be removed in a future version. Pass key as a tuple instead.
  """..."""
/gpfs/bbp.cscs.ch/home/sood/rsynced/work/project/proj83/topological-analysis-of-subvolumes/connsense/pipeline.py:244: FutureWarning: Passing lists as key for xs is deprecated and will be removed in a future version. Pass key as a tuple instead.
  
 2021-11-12 14:06:08,974: Data availabel for column R0;C10: ['nodes', 'adjacency', 'randomizations']


The nodes data, and adjacency will be as described above.
Randomizations will be return if these are available as a dataframe:

In [45]:
randomizations = data_r0c10["randomizations"]
randomizations

algorithm                            connectome  flat_x       flat_y
Erodos-Renyi-controlling-out-degree  local       3983.716857  0.0       <connsense.io.write_results.LazyMatrix object ...
Erodos-Renyi-controlling-in-degree   local       3983.716857  0.0       <connsense.io.write_results.LazyMatrix object ...
dtype: object

In this `pandas.Series` values are `LazyMatrix` instances that store a path to 
the jar in which the matrix has been pickled.
To load the data, all you have to do is

In [48]:
random_matrices = randomizations.apply(lambda m: m.matrix)

LOG.info("Randomized matrices of shapes \n%s",
         random_matrices.apply(lambda m: m.shape))

 2021-11-12 14:10:44,029: Randomized matrices of shapes 
algorithm                            connectome  flat_x       flat_y
Erodos-Renyi-controlling-out-degree  local       3983.716857  0.0       (6719, 6719)
Erodos-Renyi-controlling-in-degree   local       3983.716857  0.0       (6719, 6719)
dtype: object


Finally, let us illustrate the data formats by computing the number of 
edges by synapse class.

In [86]:
def count_edges_by_synapse_class(data, circuit, subtargets=None):
    """..."""
    subtargets = subtargets or data.get_subtargets(circuit)
    
    def count_subtarget(s):
        """..."""
        nodes = data.get_nodes(circuit, subtarget=s)
        adj = data.get_adjacency(circuit, subtarget=s, connectome="local")
        edge_nodes = np.where(pd.DataFrame.sparse.from_spmatrix(adj))
        edges = pd.DataFrame({"pre": edge_nodes[0], "post": edge_nodes[1]})
        
        pre_sc = nodes.synapse_class.iloc[edges.pre.values]
        post_sc = nodes.synapse_class.iloc[edges.post.values]
        
        return pd.DataFrame({"pre": pre_sc, "post": post_sc}).value_counts()
    
    return pd.concat([count_subtarget(s) for s in subtargets],
                    keys=subtargets, names=["subtarget"])
        

In [89]:
sample = pd.Series(tap.data.get_subtargets("Bio_M")).sample(n=10).to_list()
#q = count_edges_by_synapse_class(tap.data, circuit="Bio_M", subtargets=sample)
LOG.info("statistics for synapse_class --> synapse_class connectivity")
q.groupby(["pre", "post"]).agg(["min", "mean", "std", "median", "mad", "max"])

 2021-11-12 15:13:57,949: statistics for synapse_class --> synapse_class connectivity


min       mean           std     median         mad      max
pre post                                                                
EXC EXC   18184  3824972.0  2.059240e+06  4298252.5  1525449.20  6583985
    INH    1344   293858.1  1.615373e+05   326944.0   116566.66   559751
INH EXC    2366   303143.3  1.588469e+05   337519.5   115290.98   536854
    INH     214    23971.9  1.259928e+04    26425.0     9080.54    44276